Run with the following conda environment: `../../conda_envs/training_env`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.gridspec import GridSpec 

In [ ]:
results_folder = "../../results/simu_output/scheduling/"

In [ ]:
params = {
    'legend.fontsize': 14,
    'legend.title_fontsize': 16,
    'axes.labelsize': 18,
    'axes.titlesize': 22,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'figure.titlesize': 25,
}
plt.rc('font', size=16)
plt.rc('pdf', fonttype=42)
pylab.rcParams.update(params)

In [ ]:
coda_name = 'DUMBO'

In [ ]:
traces = ["caida", "mawi", "uni"]
models = [
    "fifo",
    "pheavy",     
    "coda", 
    "phost",
    "pfabric", 
]
nflows = [
    "10K", 
    "100K",
    "1M",
]

results = {}
for t in traces:
    results[t] = {}
    for m in models:
        results[t][m] = {}
        for n in nflows:
            try:
                df = pd.read_csv(f"{results_folder}/result_{m}_{t}_{n}.txt", names=["load", "avg_slowdown", "tmp1", "tmp2"], sep=" ")
                df = df[["load", "avg_slowdown"]]
            except FileNotFoundError:
                df = pd.DataFrame(data={
                    "load": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                    "avg_slowdown": [0., 0., 0., 0., 0., 0., 0., 0., 0.],
                })
            data = df.to_dict(orient="tight")["data"]
            results[t][m][n] = {
                i[0]: i[1] 
                for i
                in data
            }

In [ ]:
fig = plt.figure(figsize=(6,4.5))
gs = GridSpec(2, 1, height_ratios=[1, 1], hspace=0.04)
ax_load1 = fig.add_subplot(gs[0, 0])
ax_load2 = fig.add_subplot(gs[1, 0], sharex=ax_load1)

palette = {
    "coda": "tab:green",
    "pfabric" : "tab:purple",
    "fastpass": "tab:blue",
    "pheavy": "tab:red", 
    "fifo": "tab:blue",    
    "phost": "tab:orange",  
}
HATCHES = {
    "coda": None,
    "pfabric" : "--",
    "fastpass": "\\",
    "pheavy": "xx",
    "fifo": "\\\\",    
    "phost": "/",  
}

nflows = "1M"

bar_width = 0.15

load1 = 0.5
for i, t in enumerate(traces):
    for j, m in enumerate(models):
        x=i+j*bar_width-bar_width*2
        ax_load1.bar(x=x, height=results[t][m][nflows][load1], label=m, width=bar_width, color=palette[m], hatch=HATCHES[m], edgecolor="black")

ax_load1.set_yticks([1, 2, 3, 4])
ax_load1.set_xticks([])
ax_load1.text(0.98, 0.85, f'load = {load1}', color='black', size=14, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'), horizontalalignment='right', transform=ax_load1.transAxes)
ax_load1.set_axisbelow(True)

load2 = 0.8
handles_baselines, labels_baselines = [], []
for i, t in enumerate(traces):
    for j, m in enumerate(models):
        x=i+j*bar_width-bar_width*2
        ax_load2.bar(x=x, height=results[t][m][nflows][load2], label=m, width=bar_width, color=palette[m], hatch=HATCHES[m], edgecolor="black")
        if i == 2:
            handles_baselines, labels_baselines = plt.gca().get_legend_handles_labels()
            order = [0, 3, 4]

ax_load2.set_yticks([1, 3, 5, 7])
ax_load2.text(0.98, 0.85, f'load = {load2}', color='black', size=14, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'), horizontalalignment='right', transform=ax_load2.transAxes)
ax_load2.set_axisbelow(True)

handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 2]
order_baselines = [0, 3, 4]
ax_load1.legend(
    [handles[i] for i in order] + [handles_baselines[i] for i in order_baselines], 
    [labels[i].replace("coda", f"(w/ hints) {coda_name}").replace("pheavy", "(w/ hints) pHeavy 5pk") for i in order] + [labels_baselines[i].replace("fifo", "FIFO").replace("phost", "pHost").replace("pfabric", "pFabric") for i in order_baselines], 
    frameon=False, ncol=3, loc="lower center", bbox_to_anchor=(0.5, 0.99), 
    labelspacing=0.3, handletextpad=0.6, borderaxespad=.0, columnspacing=1.0
)

ax_load2.set_xticks([i for i in range(len(traces))], [t.upper() for t in traces])

ax_load1.tick_params(axis='x', which='both', bottom=False, labelbottom=False)

fig.text(0.03, 0.5, 'Normalized FCT', va='center', ha='center', rotation='vertical', fontsize=plt.rcParams['axes.labelsize'])

plt.subplots_adjust(left=0.09, right=0.99, bottom=0.09, top=0.88)
plt.savefig(f'out/scheduling_bars.pdf', format='pdf', dpi=1200)
plt.show()